# Task: Resume Matching with Job Descriptions Using PDF CVs


## Job Description Data Understanding
### and
## Candidate-Job Matching


In [ ]:
from google.colab import files
files.upload()

In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.2 MB/s eta 0:00:00


In [4]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00


In [5]:
# Load the necessary libraries
import pandas as pd
import numpy as np

import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# Load resume data
resume_data=pd.read_csv("Resume_data.csv")
resume_data.head()

,Unnamed: 0,category,candidate_resume_name,education,skills
0,0,ACCOUNTANT,10554236.pdf,northern maine community college 1994 associat...,accounting general accounting accounts payable...
1,1,ACCOUNTANT,10674770.pdf,bachelor of science accounting may 2010 univer...,highlights dba quick books mas sage software m...
2,2,ACCOUNTANT,11163645.pdf,computer applications specialist certificate p...,and attributes attributes selfmotivated and ho...
3,3,ACCOUNTANT,11759079.pdf,emory university goizueta business school 5 20...,by drafting over forty memorandums that summar...
4,4,ACCOUNTANT,12065211.pdf,bachelor of business administration accounting...,aderantcms excel quickbooks pro sql access pea...


# Job Description Data Understanding


In [7]:
# load data for Job Description Data Understanding
dataset=load_dataset("jacob-hugging-face/job-descriptions")
# load train data from job_description dataset
job_description=dataset["train"]

data=pd.DataFrame(job_description)
# Take a sample of 15 from data.
job_desc_data=data.sample(n=15,random_state=7)
job_desc_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

,company_name,job_description,position_title,description_length,model_response
638,"Dimech Services, Inc",we are seeking a detailoriented remote data en...,Data Entry Operator - REMOTE,1868,"{\n ""Core Responsibilities"": ""Compiling, cap..."
209,JobsInLogistics.com,company general transport inc\n\njob descripti...,CDL A Regional Drivers,599,"{\n ""Core Responsibilities"": ""Driving freigh..."
507,Staffing Solutions Enterprises,editor remote or onsite\n\nremote or onsite cl...,Editor (Remote or Onsite),1178,"{\n ""Core Responsibilities"": ""Provide editor..."
94,Alley,were not looking for rockstars zombies or magi...,Software Developer,4431,"{\n ""Core Responsibilities"": ""Responsibiliti..."
244,APEX Race Parts,are you a motorsport enthusiast trapped in a j...,Ecommerce Manager Remote,5540,"{\n ""Core Responsibilities"": ""Take ownership..."
789,US Parcel Service,our assistants are our frontline employees and...,Administrative Assistant - Remote / Online / W...,1874,"{\n ""Core Responsibilities"": ""Monitor employ..."
702,Intrepid Museum,intrepid museum is now hiring system administr...,System Administrator,4913,"{\n ""Core Responsibilities"": ""Ensure IT infr..."
538,DISQO,disqo is changing the way that the worlds larg...,Senior Corporate Recruiter (Contract) - (Remot...,5069,"{\n ""Core Responsibilities"": ""Manage full cy..."
797,CUSHMAN,job title\nexecutive administrative assistant ...,"Executive Administrative Assistant, Brokerage",2436,"{\n ""Core Responsibilities"": ""Meet or exceed..."
289,Swifty Web Agency,hello seekers\n\nwere a web agency based in ti...,Wordpress Developer,1122,"{\n ""Core Responsibilities"": ""Build complete..."


# Candidate-Job Matching

In [8]:
# Data Cleaning and Lemmatization
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

def clean_text(text):
    text=str(text)
    text = text.lower()
    text = re.sub(r'[^\x00-\x7f]',r' ', text)
    text = re.sub('\s+', ' ', text)
    text = "".join([word for word in text if word not in string.punctuation])
    text = re.sub("\d+", " ", text)
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text.strip()


In [9]:
# Apply data cleaning
resume_data['category'] = resume_data['category'].apply(lambda x: clean_text(x))
resume_data['education'] = resume_data['education'].apply(lambda x: clean_text(x))
resume_data['skills'] = resume_data['skills'].apply(lambda x: clean_text(x))
job_desc_data["job_description"]=job_desc_data["job_description"].apply(lambda x: clean_text(x))

In [10]:
# Combine all necessary features and create one feature
resume_data["combine_text"]=resume_data['category'] + ' ' + resume_data['education'] + ' ' + resume_data['skills']

In [11]:
resume_data.head()

,Unnamed: 0,category,candidate_resume_name,education,skills,combine_text
0,0,accountant,10554236.pdf,northern maine community college associate acc...,accounting general accounting accounts payable...,accountant northern maine community college as...
1,1,accountant,10674770.pdf,bachelor science accounting may university nor...,highlights dba quick books mas sage software m...,accountant bachelor science accounting may uni...
2,2,accountant,11163645.pdf,computer applications specialist certificate p...,attributes attributes selfmotivated honest goo...,accountant computer applications specialist ce...
3,3,accountant,11759079.pdf,emory university goizueta business school bach...,drafting forty memorandums summarized potentia...,accountant emory university goizueta business ...
4,4,accountant,12065211.pdf,bachelor business administration accounting au...,aderantcms excel quickbooks pro sql access pea...,accountant bachelor business administration ac...


In [12]:
# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [13]:
# Function to encode text using DistilBERT
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

In [14]:
# Iterate through each job description in job_desc_df
for j_index, j_row in job_desc_data.iterrows():
    job_description = j_row['job_description']

    # Encode the job description using DistilBERT
    job_desc_embedding = encode_text(job_description)
    print(j_row["position_title"])
    print(f"Job Description {j_index + 1}: {job_description}")
    print("Top 5 CVs:")

    similarity_scores = []

    # Iterate through each CV in resume_data and calculate cosine similarity
    for c_index, c_row in resume_data.iterrows():
        cv_text = c_row['combine_text']

        # Encode the CV text using DistilBERT
        cv_embedding = encode_text(cv_text)

        # Calculate cosine similarity
        similarity_score = cosine_similarity([job_desc_embedding], [cv_embedding])[0][0]
        similarity_scores.append((c_index, similarity_score))

    # Sort similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Get the top 5 CVs with the highest similarity scores
    top_5_indices = [c_index for c_index, _ in similarity_scores[:5]]

    for rank, c_index in enumerate(top_5_indices):
        c_row = resume_data.loc[c_index]
        print(f"{rank + 1}. CV {c_index + 1}: Similarity Score = {similarity_scores[rank][1]:.4f}")
        print(c_row["candidate_resume_name"])
        print(c_row['combine_text'])
    print("\n")


Data Entry Operator - REMOTE
Job Description 639: seeking detailoriented remote data entry operator compile capture maintain digital database data entry operator required compile merge format documents data entry check documents errors input data update database manage digital filing systems successful data entry operator possess excellent typing skills attentive detail skilled data entry operators able quickly identify data errors demonstrate precision work responsibilities gathering collating preparing documents materials information data entry conducting research obtain information incomplete documents materials creating digital documents paper dictation reviewing documents information accuracy informing supervisor errors inconsistencies capturing data digital databases performing regular backups updating maintaining databases archives filing systems monitoring reviewing databases correcting errors inconsistencies generating exporting data reports spreadsheets documents needed perfo